In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login
from datasets import Dataset
import pandas as pd
import torch
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

import os

login(token=os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_path = Path(str(os.getenv("MODEL_ROOT"))).resolve() / 'sft' / 'TinyLlama' / 'TinyLlama-1.1B-Chat-v1.0'
model_path = Path(str(os.getenv("MODEL_ROOT"))).resolve() / 'finetuned' / 'TinyLlama' / 'TinyLlama-1.1B-Chat-v1.0' / 'checkpoint-3'
model_path = str(model_path)

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
model.warnings_issued = {}

In [25]:
dataset_path = Path(os.getenv("DATA_ROOT")) / ".kaggle" / "cnn_dailymail"
df = pd.read_csv(str(dataset_path / "test.csv"))
dataset = Dataset.from_pandas(df)

In [6]:
print(dataset[0]['highlights'])

Experts question if  packed out planes are putting passengers at risk .
U.S consumer advisory group says minimum space must be stipulated .
Safety tests conducted on planes with more leg room than airlines offer .


In [43]:
# test
messages = [
    {
        "role": "user", 
        "content": "hello? who are you?\n" 
    }
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

output_ids = model.generate(input_ids, max_new_tokens=400)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(output_text)

<|user|>
hello? who are you?
 
<|assistant|>
Hello! I'm a chatbot that can help you with any questions you may have. To get started, simply type "hello" in the chat window and I'll respond with a greeting in return. If you have any questions, feel free to ask. I'm always here to help!


In [ ]:
def get_summary(content, model):
    message = [
        {
            "role": "user",
            "content": f"Summarize the following text in a TL;DR style in one sentence\n\n{content}\n"
        }
    ]
    prompt = tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)

    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = torch.ones_like(input_ids)
    
    output_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=500,
        do_sample=False,
        # temperature=0.7,
        top_p=0.95,
        top_k=50,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )  
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    assistant_tag = "<|assistant|>"
    if assistant_tag in output_text:
        output_text = output_text.split(assistant_tag, 1)[1].strip()

    return output_text, output_ids.shape[-1] - input_ids.shape[-1]

In [51]:
content = dataset[0]['article']
print(*(get_summary(content, model)), sep='\n')

The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


<|user|>
Summarize the following text in a TL;DR style in one sentence

Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could